# Py2TikZ Demo

This notebook demonstrates functions from the `py2tikz.py` module for converting SageMath Partition objects to strings of TikZ code for diagrams.

## Setup/Imports
This module includes the following functions, each returning a string containing TikZ commands for drawing diagrams:
* `tikzline(x1, y1, x2, y2, modifier="")` - Draw a line from $(x_{1},y_{1})$ to $(x_{2},y_{2})$, with optional `modifier` string to set draw style
* `tikznode(x, y, text, modifier="")` - Draw a node containing the text at $(x,y)$, with optional `modifier` string to set draw style
* `partition2tikz(p, x0=0, y0=0, notation='English', label_parts=False)` - Draw the Young diagram of a partition $p$
* `label_cell(i, j, text, notation='English')` - Draw a node containing the text at cell $(i,j)$ in a Young diagram. Indices are $0$-based.
* `label_diagram(p, label_function, notation='English')` - Label cells $(i,j)$ in Young diagram of $p$ according to `label_function(p,i,j)`
* `tikz_ferrers(p, notation = 'English')` - Draw the Ferrers diagram of a partition $p$
* `frobenius2tikz(coords, x0=0, y0=0, notation='English', label_parts=False)` - Draw annotated Young diagram from Frobenius coordinates
* `label_boundary(seq, col1='blue', col2='orange', notation='English')` - Label and colour partition boundary according to a $0-1$ sequence
* `writetikz2tex(tikztext, filename)` - Write TikZ diagram code to a standalone `.tex` file

In [ ]:
from py2tikz import partition2tikz, label_cell, label_diagram, tikz_ferrers, frobenius2tikz, label_boundary, writetikz2tex

## Example Usage
Below we give some examples of using the functions from the `py2tikz` module to produce strings of TikZ commands, along with the kinds of images they can produce (through combination or further modification).

In [ ]:
p = Partition([6,3,1])
print(partition2tikz(p, notation='French', label_parts=True))

![](https://edwardmpearce.github.io/tutorial-partitions/img/french_notation.png)

In [ ]:
print(label_boundary([1,0,1,1,0,1,1,1,0], notation='English'))

![](https://edwardmpearce.github.io/tutorial-partitions/img/zero_one_sequence.png)

In [ ]:
print(frobenius2tikz(([5,2,1],[4,2,0]),notation='English', label_parts=True))

![](https://edwardmpearce.github.io/tutorial-partitions/img/frobenius_coordinates.png)

Drawing examples and key non-examples of $2$-core partitions together in a single frame

In [ ]:
hook_len = lambda p, i, j : p.hook_length(i,j)
for k in range(1,5):
    p = Partition(range(k, 0, -1))
    xshift = k*(k+1)/2 - 1
    print("\\begin{{scope}}[xshift={}cm]".format(xshift))
    print(partition2tikz(p))
    print(label_diagram(p, hook_len))
    print(r"\end{scope}")
for p in [[1,1], [2]]:
    p = Partition(p)
    print(partition2tikz(p))
    print(label_diagram(p, hook_len))

Drawing key non-examples of $(3,1)$-core partitions

In [ ]:
for p in [[1,1], [3], [4,2]]:
    p = Partition(p)
    print(partition2tikz(p))
    for c in p.cells():
        print(label_cell(c[0], c[1], p.upper_hook(c[0],c[1], -1) % 3))

### Writing TikZ diagrams to .tex files for compiling into images
We calculate a list of all partitions of size $40$ which are $(5,2)$-core partitions, and then systemically produce TikZ code to draw their Young tableau with cells labelled by the quantity $$l(\square) - 2(a(\square) + 1) \pmod 5$$ (all of which will be nonzero by the core criterion). 

We write the TikZ diagram instructions to .tex files which can subsequently be compiled to .pdf or an image format such as .png for inclusion in research or teaching materials. See [here](https://github.com/edwardmpearce/tikzpictures) for more details on converting TikZ diagram instructions into .png files for portability.

In [ ]:
from abacus_extension import PartitionExt
import os
os.chdir('./texfiles')   # Change current working directory

In [ ]:
big_cores = []
for p in Partitions(40):
    p = PartitionExt(p)
    if p.is_G_core(5,2):
        big_cores.append(p)
print(len(big_cores))

In [ ]:
label_func = lambda p, i, j : p.upper_hook(i,j, -2) % 5
for p in big_cores:
    young_diagram = "\\begin{scope}\n" + partition2tikz(p) + "\n\\end{scope}"
    cell_labels = "\n\\begin{scope}[font=\\Large]\n" + label_diagram(p, label_func) + "\n\\end{scope}"
    file_id = f"5,2-core_size40_{str(p).replace(' ', '')}" # Give each TikZ diagram a unique, descriptive filename
    writetikz2tex(young_diagram + cell_labels, file_id)